# Setup

In [1]:
import torch

if 'google.colab' in str(get_ipython()):
  print('Running on Colab')
  running_on_colab = True
else:
  print('Not running on Colab')
  running_on_colab = False

if running_on_colab:
    print(torch.__version__)
    !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install -q git+https://github.com/snap-stanford/deepsnap.git
    !pip install pyarrow
    !pip install fastparquet

    from google.colab import drive
    drive.mount('/content/drive')
    filepath = '/content/drive/MyDrive/GCNN/'
    data_folder = filepath+"graph_data/"
    models_folder = filepath+"models/"
    experiments_folder = filepath+"experiments/"

else:
    data_folder = "../../data/processed/graph_data_nohubs/"
    models_folder = "../../data/models/"
    experiments_folder = "../../data/experiments/design_space_experiment/"

Running on Colab
2.0.0+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import datetime
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from torch_geometric.nn import SAGEConv, GATConv
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

In [3]:
import random
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

# Utility

In [4]:
import copy

def load_data(folder_path,load_test = False):
    if load_test:
        names = ["train","validation","test"]
    else:
        names = ["train","validation"]
    datasets = []
    for name in names:
        path = folder_path+name+".pt"
        datasets.append(torch.load(path))
    
    return datasets

def initialize_features(data,feature,dim,inplace=False):
    if inplace:
        data_object = data
    else:
        data_object = copy.copy(data)
    for nodetype, store in data_object.node_items():
        if feature == "random":
            data_object[nodetype].x = torch.rand(store["num_nodes"],dim)
        if feature == "ones":
            data_object[nodetype].x = torch.ones(store["num_nodes"],dim)
    return data_object

def save_model(model,folder_path,model_name):
    date = datetime.datetime.now()
    fdate = date.strftime("%d_%m_%y__")
    fname = f"{model_name}_{fdate}"
    torch.save(model.state_dict(), f"{folder_path}{fname}.pth")

# Load Data

In [5]:
path = data_folder+"split_dataset/"
original_train_data, original_val_data = load_data(path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model

In [6]:
from torch_geometric.nn import to_hetero

class inner_product_decoder(torch.nn.Module):
    def forward(self,x_source,x_target,edge_index,apply_sigmoid=True):
        nodes_src = x_source[edge_index[0]]
        nodes_trg = x_target[edge_index[1]]
        pred = (nodes_src * nodes_trg).sum(dim=-1)

        if apply_sigmoid:
            pred = torch.sigmoid(pred)

        return pred

class base_message_layer(torch.nn.Module):

    def __init__(self, model_params,hidden_layer=True):
        super().__init__()

        # Currently SageConv or GATConv, might have to modify this to support other Convs
        conv_type = model_params["conv_type"]
        self.conv = conv_type((-1,-1), model_params["hidden_channels"],aggr=model_params["micro_aggregation"],add_self_loops=False)
        self.normalize = model_params["L2_norm"]

        post_conv_modules = []
        if model_params["batch_norm"]:
            bn = torch.nn.BatchNorm1d(model_params["hidden_channels"])
            post_conv_modules.append(bn)
        
        if model_params["dropout"] > 0:    
            dropout = torch.nn.Dropout(p=model_params["dropout"])
            post_conv_modules.append(dropout)
        
        # No activation on final embedding layer
        if hidden_layer:
            activation = model_params["activation"]()
            post_conv_modules.append(activation)
        
        self.post_conv = torch.nn.Sequential(*post_conv_modules)

    def forward(self, x:dict, edge_index:dict) -> dict:
        x = self.conv(x,edge_index)
        x = self.post_conv(x)
        if self.normalize:
            x = torch.nn.functional.normalize(x,2,-1)
        return x

class multilayer_message_passing(torch.nn.Module):
    #TODO: consider input and output dims with skipcat. Currently the two supported convs auto-detect dimensions. Might have to modify this if i add more convs in the future.
    def __init__(self,num_layers,model_params,metadata):
        super().__init__()

        self.skip = model_params["layer_connectivity"]
        self.num_layers = num_layers

        for i in range(self.num_layers):
            hidden_layer = i != self.num_layers-1
            layer = to_hetero(base_message_layer(model_params,hidden_layer),metadata,model_params["macro_aggregation"])
            self.add_module(f"Layer_{i}",layer)
    
    def hetero_skipsum(self,x: dict, x_i:dict) -> dict:
        x_transformed = {}
        for key,x_val in x.items():
            x_i_val = x_i[key]
            transformed_val = x_val + x_i_val
            x_transformed[key] = transformed_val

        return x_transformed

    def hetero_skipcat(self,x: dict, x_i:dict) -> dict:
        x_transformed = {}
        for key,x_val in x.items():
            x_i_val = x_i[key]
            transformed_val = torch.cat([x_val,x_i_val],dim=-1)
            x_transformed[key] = transformed_val

        return x_transformed
    
    def forward(self, x:dict, edge_index:dict) -> dict:
        for i, layer in enumerate(self.children()):
            x_i = x
            x = layer(x,edge_index)
            if self.skip == "skipsum":
                x = self.hetero_skipsum(x,x_i)
            elif self.skip == "skipcat" and i < self.num_layers -1:
                x = self.hetero_skipcat(x,x_i)
        
        return x 

class MLP(torch.nn.Module):
    def __init__(self,num_layers,in_dim,out_dim,model_params,hidden_dim=None):
        super().__init__()

        hidden_dim = out_dim if hidden_dim is None else hidden_dim
        
        modules = []
        if num_layers == 1:
            modules.append(torch.nn.Linear(in_dim,out_dim))
        else:
            for i in range(num_layers):
                final_layer = i == num_layers-1
                first_layer = i == 0
                if first_layer:
                    modules.append(torch.nn.Linear(in_dim,hidden_dim))
                    modules.append(model_params["activation"]())
                elif final_layer:
                    modules.append(torch.nn.Linear(hidden_dim,out_dim))
                else:
                    modules.append(torch.nn.Linear(hidden_dim,hidden_dim))
                    modules.append(model_params["activation"]())
        
        self.model = torch.nn.Sequential(*modules)
    
    def forward(self,x):
        x = self.model(x)
        return x

class base_encoder(torch.nn.Module):
    def __init__(self,model_params,metadata):
        super().__init__()

        self.has_pre_mlp = model_params["pre_process_layers"] > 0
        self.has_post_mlp = model_params["post_process_layers"] > 0

        if self.has_pre_mlp:
            self.pre_mlp = to_hetero(MLP(model_params["pre_process_layers"],model_params["feature_dim"],model_params["hidden_channels"],model_params),metadata)
        
        self.message_passing = multilayer_message_passing(model_params["msg_passing_layers"],model_params,metadata)

        if self.has_post_mlp:
            self.post_mlp = to_hetero(MLP(model_params["post_process_layers"],model_params["hidden_channels"],model_params["hidden_channels"],model_params),metadata)
    
    def forward(self,x:dict,edge_index:dict) -> dict :
        if self.has_pre_mlp:
            x = self.pre_mlp(x)

        x = self.message_passing(x,edge_index)
        
        if self.has_post_mlp:
            x = self.post_mlp(x)

        return x

class base_model(torch.nn.Module):
    def __init__(self, model_params,metadata):
        super().__init__()

        default_model_params = {
            "hidden_channels":32,
            "conv_type":SAGEConv,
            "batch_norm": True,
            "dropout":0,
            "activation":torch.nn.LeakyReLU,
            "micro_aggregation":"mean",
            "macro_aggregation":"mean",
            "layer_connectivity":None,
            "L2_norm":False,
            "feature_dim": 10,
            "pre_process_layers":0,
            "msg_passing_layers":2,
            "post_process_layers":0,
        }
        
        for arg in default_model_params:
            if arg not in model_params:
                model_params[arg] = default_model_params[arg]
        
        self.encoder = base_encoder(model_params,metadata)
        self.decoder = inner_product_decoder()
        self.loss_fn = torch.nn.BCELoss()
    
    def decode(self,x:dict,edge_label_index:dict,supervision_types):
        pred_dict = {}
        for edge_type in supervision_types:
            edge_index = edge_label_index[edge_type]

            src_type = edge_type[0]
            trg_type = edge_type[2]

            x_src = x[src_type]
            x_trg = x[trg_type]

            pred = self.decoder(x_src,x_trg,edge_index)

            pred_dict[edge_type] = pred
        
        return pred_dict
    
    def forward(self,data,supervision_types):
        x = data.x_dict
        adj_t = data.adj_t_dict
        edge_label_index = data.edge_label_index_dict

        x = self.encoder(x,adj_t)
        pred = self.decode(x,edge_label_index,supervision_types)
        return pred
    
    def loss(self, prediction_dict, label_dict):
        loss = 0
        num_types = len(prediction_dict.keys())
        for edge_type,pred in prediction_dict.items():
            y = label_dict[edge_type]
            loss += self.loss_fn(pred, y.type(pred.dtype))
        return loss/num_types

# Train and eval functions

In [7]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

@torch.no_grad()
def hits_at_k(y_true,x_prob,k,key) -> dict:
    """Dados los tensores x_prob y edge_label, calcula cuantas predicciones hizo correctamente en los primeros k puntajes.
    x_prob es la predicción del modelo luego de aplicar sigmoid (sin redondear, osea, el puntaje crudo)"""

    #ordeno los puntajes de mayor a menor
    x_prob, indices = torch.sort(x_prob, descending=True)

    #me quedo solo con los k mayor punteados
    x_prob = x_prob[:k]
    indices = indices[:k]

    if any(x_prob < 0.5):
      threshold_index = (x_prob < 0.5).nonzero()[0].item()
      print(f"Top {k} scores for {key} below classification threshold 0.5, threshold index: {threshold_index}")

    #busco que label tenían esas k preds
    labels = y_true[indices]

    #cuento cuantas veces predije uno positivo en el top k
    hits = labels.sum().item()

    return hits

def train(model, optimizer, graph,supervision_types):
    model.train()
    optimizer.zero_grad()
    preds = model(graph,supervision_types)
    edge_label = graph.edge_label_dict
    loss = model.loss(preds, edge_label)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def get_val_loss(model,val_data,supervision_types):
    model.eval()
    preds = model(val_data,supervision_types)
    edge_label = val_data.edge_label_dict
    loss = model.loss(preds, edge_label)

    return loss.item()

def get_metrics(y_true, x_pred):
   acc = round(accuracy_score(y_true,x_pred),2)
   ap = round(average_precision_score(y_true, x_pred),2)
   roc_auc = round(roc_auc_score(y_true,x_pred),2)

   return acc,ap ,roc_auc
  
@torch.no_grad()
def test(model,data,supervision_types,metric):
  model.eval()
  preds = model(data,supervision_types)
  edge_label = data.edge_label_dict
  all_preds = []
  all_true = []
  for key,pred in preds.items():
      pred_label = torch.round(pred)
      ground_truth = edge_label[key]
      all_preds.append(pred_label)
      all_true.append(ground_truth)
  total_predictions = torch.cat(all_preds, dim=0).cpu().numpy()
  total_true = torch.cat(all_true, dim=0).cpu().numpy()
  score = metric(total_true,total_predictions)
  return score
  

@torch.no_grad()
def full_test(model,data,supervision_types,k,global_score=True):
  model.eval()
  preds = model(data,supervision_types)
  edge_label = data.edge_label_dict
  metrics = {}

  if global_score:
    all_scores = []
    all_preds = []
    all_true = []
    for key,pred in preds.items():
        pred_label = torch.round(pred)
        ground_truth = edge_label[key]
        all_scores.append(pred)
        all_preds.append(pred_label)
        all_true.append(ground_truth)

    total_predictions = torch.cat(all_preds, dim=0)
    total_true = torch.cat(all_true, dim=0)
    total_scores = torch.cat(all_scores,dim=0)

    acc, ap, roc_auc =  get_metrics(total_true.cpu().numpy(), total_predictions.cpu().numpy())
    hits_k = hits_at_k(total_true,total_scores,k,"all")
    metrics["all"] = [acc,ap,roc_auc,hits_k]

  else:
    for key,pred in preds.items():
        pred_label = torch.round(pred)
        ground_truth = edge_label[key]
        acc, ap, roc_auc = get_metrics(ground_truth.cpu().numpy(), pred_label.cpu().numpy())
        hits_k = hits_at_k(ground_truth,pred,k,key)
        metrics[key] = [acc,ap, roc_auc,hits_k]
  
  return metrics

In [8]:
def plot_training_stats(title, train_losses,val_losses, train_metric,val_metric,metric_str):

  fig, ax = plt.subplots(figsize=(8,5))
  ax2 = ax.twinx()

  ax.set_xlabel("Training Epochs")
  ax2.set_ylabel(metric_str)
  ax.set_ylabel("Loss")

  plt.title(title)
  p1, = ax.plot(train_losses, "b-", label="training loss")
  p2, = ax2.plot(val_metric, "r-", label=f"val {metric_str}")
  p3, = ax2.plot(train_metric, "o-", label=f"train {metric_str}")
  p4, = ax.plot(val_losses,"b--",label=f"validation loss")
  plt.legend(handles=[p1, p2, p3,p4],loc=2)
  plt.show()

# Experiment

Me guardo los parametros, las 4 curvas de entrenamiento y el score.

In [9]:
# experiment_params = {
#     "hidden_channels":[32,64,128],
#     "conv_type":[SAGEConv,GATConv],
#     "batch_norm": [True],
#     "dropout":[0.1],
#     "activation":[torch.nn.LeakyReLU],
#     "micro_aggregation":["mean","sum","max","concat"],
#     "macro_aggregation":["mean","sum","max","concat"],
#     "layer_connectivity":[None,"skipsum","skipcat"],
#     "L2_norm":[True,False],
#     "pre_process_layers":[0,1,2],
#     "msg_passing_layers":[1,2,3],
#     "post_process_layers":[0,1,2],
#     "feature_dim":[10,50,100,200],
#     "feature_type":["ones","random"],
#     'weight_decay': [1e-5,1e-3,0],
#     'lr': [0.01,0.001,0.1],
#     'epochs':[400],
#     "patience":[2,5,10],
#     "delta":[0.1,0.01,0.5]
# }

In [10]:
from sklearn.model_selection import ParameterGrid
supervision_types = [('gene_protein', 'gda', 'disease')]
def perform_hyperparameter_search(param_grid, train_set, val_set):
  default = {
      "hidden_channels":[32],
      "conv_type":[SAGEConv],
      "batch_norm": [True],
      "dropout":[0.1],
      "activation":[torch.nn.LeakyReLU],
      "micro_aggregation":["mean"],
      "macro_aggregation":["mean"],
      "layer_connectivity":[None],
      "L2_norm":[False],
      "feature_dim": ["random"],
      "pre_process_layers":[0],
      "msg_passing_layers":[2],
      "post_process_layers":[0],
      "feature_dim":[10],
      "feature_type":["random"],
      'weight_decay': [1e-3],
      'lr': [0.001],
      'epochs':[400],
      "patience":[10],
      "delta":[0.1]
  }

  for arg in default:
    if arg not in param_grid:
      param_grid[arg] = default[arg]

  grid = ParameterGrid(param_grid)

  auc_results = []
  models = []

  for eid,params in enumerate(grid):
    # Launch a training experiment using the current set of parameters
    val_auc,current_model,curve_data = launch_experiment(
                   params,
                   train_set,
                   val_set)
    
    params["auc"] = val_auc
    params["curve_data"] = curve_data

    auc_results.append(params)
    models.append(current_model)

    print(f"Validation AUC: {round(val_auc,2)}. Iteration: {eid+1} of {grid.__len__()}")

  return auc_results, models

def launch_experiment(params, train_set, val_set):
  
  # Construct the features to be used.
  train_set = initialize_features(train_set,params["feature_type"],params["feature_dim"])
  val_set = initialize_features(val_set,params["feature_type"],params["feature_dim"])
  train_set.to(device)
  val_set.to(device)

  #Initialize model
  model = base_model(params,train_set.metadata())
  model = model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params["weight_decay"])
  train_losses = []
  val_losses = []
  train_scores = []
  val_scores = []

  metric = roc_auc_score
  epochs = params["epochs"]

  early_stopper = EarlyStopper(params["patience"],params["delta"])
  for epoch in range(epochs):
      train_loss = train(model,optimizer,train_set,supervision_types)
      val_loss = get_val_loss(model,val_set,supervision_types)
      train_score = test(model,train_set,supervision_types,metric)
      val_score = test(model,val_set,supervision_types,metric)

      train_losses.append(train_loss)
      train_scores.append(train_score)
      val_scores.append(val_score)
      val_losses.append(val_loss)
      
      if early_stopper.early_stop(val_loss):
          print("Early stopping")
          break

  val_auc = test(model,val_set,supervision_types,roc_auc_score)
  curve_data = [train_losses,val_losses,train_scores,val_scores]

  return val_auc, model, curve_data

## Run multiple experiments

In [11]:
params_1 = {
    "hidden_channels":[32,64,128],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_2 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv,GATConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_3 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["mean","sum","max"],
    "macro_aggregation":["mean","sum","max"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_4 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None,"skipcat","skipsum"],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_5 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True,False],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_6 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[0,1,2],
    "msg_passing_layers":[2],
    "post_process_layers":[0,1,2],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_7 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[1,2,3],
    "post_process_layers":[1],
    "feature_dim":[10],
    "feature_type":["random"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

params_8 = {
    "hidden_channels":[32],
    "conv_type":[SAGEConv],
    "batch_norm": [True],
    "dropout":[0.1],
    "activation":[torch.nn.LeakyReLU],
    "micro_aggregation":["sum"],
    "macro_aggregation":["mean"],
    "layer_connectivity":[None],
    "L2_norm":[True],
    "pre_process_layers":[1],
    "msg_passing_layers":[2],
    "post_process_layers":[1],
    "feature_dim":[10,50,100],
    "feature_type":["random","ones"],
    'weight_decay': [1e-3],
    'lr': [0.001],
    'epochs':[400],
    "patience":[5],
    "delta":[0.01]
}

param_list = [params_1,params_2,params_3,params_4,params_5,params_6,params_7,params_8]
#param_list = [params_5]
all_results = []
all_models = []

for i,param in enumerate(param_list):
    print(f"Experiment {i+1} of {len(param_list)}")
    experiment_results, models = perform_hyperparameter_search(param, original_train_data, original_val_data)
    results_df = pd.DataFrame(experiment_results).astype({"activation":str,"conv_type":str})
    all_results.append(results_df)
    all_models.append(models)

final_results = pd.concat(all_results).reset_index(drop=True)
final_models = list(itertools.chain(*all_models))


date = datetime.datetime.now()
fdate = date.strftime("%d_%m_%y")
fname = experiments_folder+"experiment_"+fdate+".parquet"
final_results.to_parquet(fname)

for i,model in enumerate(final_models):
    model_name = f"experiment_{i}"
    save_model(model,experiments_folder,model_name)

Experiment 1 of 8
Early stopping
Validation AUC: 0.89. Iteration: 1 of 3
Early stopping
Validation AUC: 0.89. Iteration: 2 of 3
Early stopping
Validation AUC: 0.89. Iteration: 3 of 3
Experiment 2 of 8
Early stopping
Validation AUC: 0.89. Iteration: 1 of 2
Early stopping
Validation AUC: 0.82. Iteration: 2 of 2
Experiment 3 of 8
Early stopping
Validation AUC: 0.86. Iteration: 1 of 9
Early stopping
Validation AUC: 0.88. Iteration: 2 of 9
Early stopping
Validation AUC: 0.89. Iteration: 3 of 9
Early stopping
Validation AUC: 0.86. Iteration: 4 of 9
Early stopping
Validation AUC: 0.9. Iteration: 5 of 9
Early stopping
Validation AUC: 0.88. Iteration: 6 of 9
Early stopping
Validation AUC: 0.87. Iteration: 7 of 9
Early stopping
Validation AUC: 0.89. Iteration: 8 of 9
Early stopping
Validation AUC: 0.89. Iteration: 9 of 9
Experiment 4 of 8
Early stopping
Validation AUC: 0.89. Iteration: 1 of 3
Early stopping
Validation AUC: 0.9. Iteration: 2 of 3
Early stopping
Validation AUC: 0.89. Iteration: 3 

In [14]:
final_results.sort_values(by="auc", ascending=False)

,L2_norm,activation,batch_norm,conv_type,delta,dropout,epochs,feature_dim,feature_type,hidden_channels,...,lr,macro_aggregation,micro_aggregation,msg_passing_layers,patience,post_process_layers,pre_process_layers,weight_decay,auc,curve_data
34,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,50,ones,32,...,0.001,mean,sum,2,5,1,1,0.001,0.905606,"[[0.7131608724594116, 0.6851040124893188, 0.66..."
36,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,100,ones,32,...,0.001,mean,sum,2,5,1,1,0.001,0.903642,"[[0.7183493971824646, 0.6900052428245544, 0.67..."
32,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,ones,32,...,0.001,mean,sum,2,5,1,1,0.001,0.899238,"[[0.717365562915802, 0.6935465931892395, 0.673..."
15,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,mean,sum,2,5,1,1,0.001,0.897334,"[[0.6668938398361206, 0.6482419371604919, 0.63..."
9,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,sum,sum,2,5,1,1,0.001,0.896560,"[[0.7035208344459534, 0.6791146993637085, 0.66..."
25,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,mean,sum,2,5,2,0,0.001,0.896203,"[[0.6906418204307556, 0.685053288936615, 0.679..."
30,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,mean,sum,3,5,1,1,0.001,0.895191,"[[0.6793730854988098, 0.6587333083152771, 0.64..."
24,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,mean,sum,2,5,1,2,0.001,0.895012,"[[0.649621844291687, 0.6250196099281311, 0.609..."
1,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,64,...,0.001,mean,sum,2,5,1,1,0.001,0.893108,"[[0.6938392519950867, 0.6633586883544922, 0.63..."
27,True,<class 'torch.nn.modules.activation.LeakyReLU'>,True,<class 'torch_geometric.nn.conv.sage_conv.SAGE...,0.01,0.1,400,10,random,32,...,0.001,mean,sum,2,5,2,2,0.001,0.893108,"[[0.6928339600563049, 0.681195855140686, 0.672..."
